# Capstone Project - The Battle of the Neighborhoods

### Applied Data Science Capstone by IBM/Coursera

In [76]:
import numpy as np 
import pandas as pd
import requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents



# Tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print("Libraries imported.")

Libraries imported.


In [80]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


Solving environment: / ^C
failed

CondaError: KeyboardInterrupt



In [81]:
!pip install folium
import folium # plotting library

print('Folium installed')

Folium installed


#### Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [82]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [83]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data.content, 'html.parser')

#### Creating dataframe consist of three columns: PostalCode, Borough, and Neighborhood

In [84]:
table = soup.find('tbody')
row = table.select('tr')
rec = [i.get_text() for i in row]

df = pd.DataFrame(rec)

#### Data Cleansing

In [85]:
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,,
2,,M2A,,Not assigned,,,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,Regent Park / Harbourfront,


#### Ignoring cells with a borough that is Not assigned

In [86]:
df4 = df3[df3.Borough != 'Not assigned']
df4

,,Postal code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,Regent Park / Harbourfront,
6,,M6A,,North York,,Lawrence Manor / Lawrence Heights,
7,,M7A,,Downtown Toronto,,Queen's Park / Ontario Provincial Government,
9,,M9A,,Etobicoke,,Islington Avenue,
10,,M1B,,Scarborough,,Malvern / Rouge,
12,,M3B,,North York,,Don Mills,
13,,M4B,,East York,,Parkview Hill / Woodbine Gardens,
14,,M5B,,Downtown Toronto,,"Garden District, Ryerson",


#### Assigning Borough to "Not Assigned" Neigborhood

In [87]:
# For Neighborhood="Not assigned", assign the value the same as Borough
for index, row in df4.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

df4.head()

,,Postal code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,Regent Park / Harbourfront,
6,,M6A,,North York,,Lawrence Manor / Lawrence Heights,
7,,M7A,,Downtown Toronto,,Queen's Park / Ontario Provincial Government,


In [88]:
df6 = df4.rename(columns={'Postal code': 'PostalCode'}) #Renaming Postal Code
df6.head()  #Saving Raw Data

,,PostalCode,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,Regent Park / Harbourfront,
6,,M6A,,North York,,Lawrence Manor / Lawrence Heights,
7,,M7A,,Downtown Toronto,,Queen's Park / Ontario Provincial Government,


In [89]:
df_drop_neigh = df6.drop('Neighborhood', 1)
df_drop_neigh.head()

,,PostalCode,,Borough,,
3,,M3A,,North York,,
4,,M4A,,North York,,
5,,M5A,,Downtown Toronto,,
6,,M6A,,North York,,
7,,M7A,,Downtown Toronto,,


#### Spliting Neighborhood Data 

In [92]:
df_tmp = df6.copy()

df_split = pd.DataFrame(df_tmp.Neighborhood.str.split('/').tolist(), index=df_tmp.PostalCode).stack()
df_split = df_split.reset_index([0, 'PostalCode'])
df_split.columns = ['PostalCode','Neighborhood']

df_split


,PostalCode,Neighborhood
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,Regent Park
3,M5A,Harbourfront
4,M6A,Lawrence Manor
5,M6A,Lawrence Heights
6,M7A,Queen's Park
7,M7A,Ontario Provincial Government
8,M9A,Islington Avenue
9,M1B,Malvern


In [93]:
df_tab = pd.merge(df_drop_neigh, df_split, on='PostalCode')
df_tab

,,PostalCode,,Borough,,,Neighborhood
0,,M3A,,North York,,,Parkwoods
1,,M4A,,North York,,,Victoria Village
2,,M5A,,Downtown Toronto,,,Regent Park
3,,M5A,,Downtown Toronto,,,Harbourfront
4,,M6A,,North York,,,Lawrence Manor
5,,M6A,,North York,,,Lawrence Heights
6,,M7A,,Downtown Toronto,,,Queen's Park
7,,M7A,,Downtown Toronto,,,Ontario Provincial Government
8,,M9A,,Etobicoke,,,Islington Avenue
9,,M1B,,Scarborough,,,Malvern


#### Print the number of rows of your dataframe

In [94]:
df_tab.shape

(208, 7)

In [95]:
df_postalcode =pd.DataFrame( df_tab.groupby(['PostalCode','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_postalcode.reset_index(inplace=True)
df_postalcode

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


#### Importing geographical coordinates of each postal code

In [96]:
url = "http://cocl.us/Geospatial_data"
df7 = pd.read_csv(url)
df7.rename(columns={'Postal Code': 'PostalCode'}, inplace=True) # chane the first column's name to be as the first dataframe
df7.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Combine Dataframe

In [97]:
df_merge = pd.merge(df_postalcode, df7, on='PostalCode')
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [98]:
## Saving the file
df_merge.to_csv('Toronto_Postalcode_neighborhood.csv')

In [99]:
df_data = pd.read_csv('Toronto_Postalcode_neighborhood.csv', index_col=0)


## Finding number of Borough & Neighborhood in the dataframe

print('The dataframe has {} Borough and {} Neighborhood.'.format(
        len(df_data['Borough'].unique()),
        df_data.shape[0]
    )
)



The dataframe has 10 Borough and 103 Neighborhood.


In [100]:
df_data.groupby('Borough').count()['Neighborhood']


Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

#### Filtering Data based on Toronto

In [101]:
df_toronto=df_data[df_data['Borough'].str.contains('Toronto')]

print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


#### Fetch Geographical coodinate of Toronto

In [102]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [103]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Creating a map where Borough contains only Toronto

In [104]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill_color='green',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Use of Foursquare API to explore the neighborhoods

In [105]:
CLIENT_ID = 'BFNBDSDWET3P03IASXXA0IIWZJ5U20LRKAW2I1PXLW5KOR05'
CLIENT_SECRET = 'P2JYOLTXU5YUISQ5POOINUZGPE5MLLAVOLJPYT1DZMI05QYS'
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BFNBDSDWET3P03IASXXA0IIWZJ5U20LRKAW2I1PXLW5KOR05
CLIENT_SECRET:P2JYOLTXU5YUISQ5POOINUZGPE5MLLAVOLJPYT1DZMI05QYS


#### Top 100 venues within a radius of 500 meters.

In [106]:
radius = 500
LIMIT = 100

venues = []

for latitude, longitude, post, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['PostalCode'], df_toronto['Borough'], df_toronto['Neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
  
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            neighborhood,
            latitude, 
            longitude, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        

venues_df = pd.DataFrame(venues)
venues_df.head()


,0,1,2,3,4,5,6,7
0,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [107]:
## Convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

## Define the column Names
venues_df.columns = ['PostalCode',
                     'Neighborhood', 
                     'Neighborhood_Latitude', 
                     'Neighborhood_Longitude', 
                     'Venue', 
                     'Venue_Latitude', 
                     'Venue_Longitude', 
                     'Venue_Category']
    
                     
print(venues_df.shape)


(1604, 8)


In [108]:
venues_df.head()

,PostalCode,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [109]:
## Find Number of venues for each PostalCode
venues_df.groupby(["PostalCode","Neighborhood"]).count()

,,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
PostalCode,Neighborhood,,,,,,
M4E,The Beaches,5,5,5,5,5,5
M4K,"The Danforth West , Riverdale",42,42,42,42,42,42
M4L,"India Bazaar , The Beaches West",20,20,20,20,20,20
M4M,Studio District,40,40,40,40,40,40
M4N,Lawrence Park,3,3,3,3,3,3
M4P,Davisville North,8,8,8,8,8,8
M4R,North Toronto West,18,18,18,18,18,18
M4S,Davisville,33,33,33,33,33,33
M4T,"Moore Park , Summerhill East",1,1,1,1,1,1


In [110]:
## Number of unique Venue Category
print('There are {} uniques categories.'.format(len(venues_df['Venue_Category'].unique())))

There are 228 uniques categories.


In [112]:
## Some Examples
venues_df['Venue_Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Coffee Shop',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Bakery',
       'Caribbean Restaurant', 'Café', 'Indian Restaurant',
       'Japanese Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Liquor Store',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Movie Theater',
       'Sandwich Place', 'Light Rail Station', 'Fish Market', 'Gay Bar',
       'Seafood Restaurant', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store', 'Wine Bar',
       'Thai Restaurant', 'Coworking Space', 'Latin American Restaurant',


#### Checking Indian Restaurant in the fetched list or not

In [113]:
"Indian Restaurant" in venues_df['Venue_Category'].unique()

True

In [114]:
## Analyzing Area Based on Category

df_cat = pd.get_dummies(venues_df[['Venue_Category']], prefix="", prefix_sep="")

# Add postal, borough and neighborhood column back to dataframe
df_cat['Neighborhood'] = venues_df['Neighborhood'] 

# Move postal, borough and neighborhood column to the first column
#fixed_columns = list(df_cat.columns[-3:]) + list(df_cat.columns[:-3])
#df_cat = df_cat[fixed_columns]

# Move PostalCode, Neighborhood column to the first column
fixed_columns = [df_cat.columns[-1]] + list(df_cat.columns[:-1])
df_cat = df_cat[fixed_columns]

print(df_cat.shape)
df_cat.head()



(1604, 228)


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
## Calculating Mean

df_cat_mean = df_cat.groupby(["Neighborhood"]).mean().reset_index()

print(df_cat_mean.shape)
df_cat_mean.head()

(39, 228)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower , King and Spadina , Railway Lands ...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.016667,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0


In [116]:
len(df_cat_mean[df_cat_mean["Indian Restaurant"] > 0])

7

In [117]:
df_indian = df_cat_mean[["Neighborhood","Indian Restaurant"]]
df_indian.head()

,Neighborhood,Indian Restaurant
0,Berczy Park,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000
2,Business reply mail Processing CentrE,0.000000
3,"CN Tower , King and Spadina , Railway Lands ...",0.000000
4,Central Bay Street,0.016667


#### Clustering

In [118]:
from sklearn.cluster import KMeans
totclusters = 3

df_cluster = df_indian.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=totclusters, random_state=1)
kmeans.fit_transform(df_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
      dtype=int32)

In [119]:
df_tor_merged = df_indian.copy()

# Add clustering labels
df_tor_merged["Cluster_Labels"] = kmeans.labels_
df_tor_merged.head()

,Neighborhood,Indian Restaurant,Cluster_Labels
0,Berczy Park,0.000000,0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0
2,Business reply mail Processing CentrE,0.000000,0
3,"CN Tower , King and Spadina , Railway Lands ...",0.000000,0
4,Central Bay Street,0.016667,1


In [51]:
#df_tor_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
#df_tor_merged.head(5)

,Neighborhood,Indian Restaurant,Cluster_Labels,PostalCode,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [120]:
df_tor_merged = df_tor_merged.join(venues_df.set_index("Neighborhood"), on="Neighborhood")

print(df_tor_merged.shape)
df_tor_merged.head()

(1604, 10)


,Neighborhood,Indian Restaurant,Cluster_Labels,PostalCode,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,M5E,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [121]:
## Dropping Postal Code
df_list=df_tor_merged.drop(['PostalCode'], axis=1)
df_list.head()

,Neighborhood,Indian Restaurant,Cluster_Labels,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [126]:
df_list.sort_values(["Cluster_Labels"], inplace=True)
df_list.head()

,Neighborhood,Indian Restaurant,Cluster_Labels,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
24,"Regent Park , Harbourfront",0.0,0,43.654260,-79.360636,Sumach-Shuter Parkette,43.658490,-79.358692,Playground
24,"Regent Park , Harbourfront",0.0,0,43.654260,-79.360636,Residence & Conference Centre,43.653040,-79.357040,Hotel
24,"Regent Park , Harbourfront",0.0,0,43.654260,-79.360636,The Healthy Road,43.656265,-79.357119,Health Food Store
24,"Regent Park , Harbourfront",0.0,0,43.654260,-79.360636,FUEL+,43.653193,-79.355867,Coffee Shop


In [127]:
map_clusters = folium.Map(location=[latitude, longitude],zoom_start=20)

## Add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'gray'
markers_colors[3] = 'orange'
markers_colors[4] = 'green'
markers_colors[5] = 'yellow'
for lat, lon, cluster in zip(df_list['Neighborhood_Latitude'], df_list['Neighborhood_Longitude'], df_list['Cluster_Labels']):
    
    folium.CircleMarker(
        [lat, lon], radius=5,
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [135]:
## Cluster 0
df_list.loc[(df_list['Cluster_Labels'] ==0) & (df_list['Venue_Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster_Labels,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category


### Cluster 1

In [131]:
## Cluster 1
df_list.loc[(df_list['Cluster_Labels'] ==1) & (df_list['Venue_Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster_Labels,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
36,"The Danforth West , Riverdale",0.023810,1,43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant
14,"Harbourfront East , Union Station , Toronto ...",0.010000,1,43.640816,-79.381752,Indian Roti House,43.639060,-79.385422,Indian Restaurant
30,"St. James Town , Cabbagetown",0.022222,1,43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
4,Central Bay Street,0.016667,1,43.657952,-79.387383,Colaba Junction,43.660940,-79.385635,Indian Restaurant
6,Church and Wellesley,0.012987,1,43.665860,-79.383160,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant


### Cluster 2

In [130]:
## Cluster 2
df_list.loc[(df_list['Cluster_Labels'] ==2) & (df_list['Venue_Category'] == 'Indian Restaurant') ]

,Neighborhood,Indian Restaurant,Cluster_Labels,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
8,Davisville,0.030303,2,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
34,"The Annex , North Midtown , Yorkville",0.041667,2,43.672710,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
